In [1]:
#The file is cleaned from the original data set. Take R1work as an example, and other activities are handled in exactly the same way.
#import libraries
import pandas as pd
import numpy as np
import scipy
import datetime
import numexpr
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn.metrics import mean_squared_error, r2_score
from math import * 

In [2]:
#read data
data_raw=pd.read_csv('data/data.csv')

In [3]:
data_raw.dtypes

Datetime    object
Sensor      object
State       object
Activity    object
Note        object
dtype: object

In [4]:
data_raw

,Datetime,Sensor,State,Activity,Note
0,2009-02-02 07:15:17,M35,ON,R1_Bed_to_Toilet,begin
1,2009-02-02 07:15:21,AD1-A,2.82231,NaN,NaN
2,2009-02-02 07:15:23,M35,OFF,NaN,NaN
3,2009-02-02 07:15:23,M35,ON,NaN,NaN
4,2009-02-02 07:15:28,AD1-A,2.79926,NaN,NaN
...,...,...,...,...,...
138625,2009-04-04 07:26:16,M48,OFF,NaN,NaN
138626,2009-04-04 07:27:47,M48,ON,NaN,NaN
138627,2009-04-04 07:27:50,M47,ON,NaN,NaN
138628,2009-04-04 07:27:59,M47,OFF,NaN,NaN


In [5]:
#value count
df=data_raw['Activity'].value_counts()

In [6]:
df

Meal_Preparation          235
R1_Work                   118
R1_Personal_Hygiene        90
R2_Work                    87
R2_Bed_to_Toilet           78
R2_Personal_Hygiene        78
R1_Sleep                   70
R2_Sleep                   70
R1_Bed_to_Toilet           68
Watch_TV                   62
R1_Bed_to_Toilet_begin     26
R1_Bed_to_Toilet_end       26
Study                      18
Clean                       4
Wash_Bathtub                2
Name: Activity, dtype: int64

In [7]:
group = data_raw.groupby('Activity')
group.describe()

Datetime                                  Sensor  \
                          count unique                  top freq  count   
Activity                                                                  
Clean                         4      4  2009-02-06 17:16:01    1      4   
Meal_Preparation            235    234  2009-04-02 08:13:03    2    235   
R1_Bed_to_Toilet             68     68  2009-02-02 07:15:17    1     68   
R1_Bed_to_Toilet_begin       26     26  2009-02-16 23:19:23    1     26   
R1_Bed_to_Toilet_end         26     26  2009-02-16 23:23:11    1     26   
R1_Personal_Hygiene          90     90  2009-02-02 07:21:04    1     90   
R1_Sleep                     70     70  2009-02-02 22:10:55    1     70   
R1_Work                     118    118  2009-02-02 14:29:03    1    118   
R2_Bed_to_Toilet             78     78  2009-02-02 07:44:13    1     78   
R2_Personal_Hygiene          78     78  2009-02-02 10:51:07    1     78   
R2_Sleep                     70     70  2009-02-02 22:29:19    1     70   
R2_Work                      87     87  2009-02-02 15:23:23    1     87   
Study                        18     18  2009-02-06 20:01:30    1     18   
Wash_Bathtub                  2      2  2009-02-04 19:41:46    1      2   
Watch_TV                     62     62  2009-02-02 11:41:34    1     62   

                                        State                   Note         \
                       unique  top freq count unique  top freq count unique   
Activity                                                                      
Clean                       4  M19    1     4      2   ON    2     4      2   
Meal_Preparation           12  M17   73   235     28   ON  127   235      2   
R1_Bed_to_Toilet           14  M36   18    68      2   ON   50    68      2   
R1_Bed_to_Toilet_begin      3  M38   13    26      1   ON   26     0      0   
R1_Bed_to_Toilet_end        1  M32   26    26      1   ON   26     0      0   
R1_Personal_Hygiene        12  M36   22    90      2   ON   62    90      2   
R1_Sleep                    6  M35   49    70      2  OFF   36    70      2   
R1_Work                     8  M31   70   118      2  OFF   60   118      2   
R2_Bed_to_Toilet           13  M50   16    78      2   ON   57    78      2   
R2_Personal_Hygiene        12  M50   19    78      2   ON   56    78      2   
R2_Sleep                    5  M48   36    70      2   ON   36    70      2   
R2_Work                     7  M44   41    87      2  OFF   44    87      2   
Study                       7  M13    6    18      2  OFF   11    18      2   
Wash_Bathtub                2  M48    1     2      2   ON    1     2      2   
Watch_TV                   12  M08   15    62      4   ON   41    62      2   

                                    
                          top freq  
Activity                            
Clean                   begin    2  
Meal_Preparation          end  118  
R1_Bed_to_Toilet        begin   34  
R1_Bed_to_Toilet_begin    NaN  NaN  
R1_Bed_to_Toilet_end      NaN  NaN  
R1_Personal_Hygiene     begin   45  
R1_Sleep                begin   35  
R1_Work                   end   60  
R2_Bed_to_Toilet        begin   39  
R2_Personal_Hygiene     begin   39  
R2_Sleep                begin   35  
R2_Work                 begin   44  
Study                   begin    9  
Wash_Bathtub            begin    1  
Watch_TV                begin   31

In [8]:
#change data set
groups = {}
for k, g in group:
    groups[k]=g  
for i in groups:
    groups[i]=pd.DataFrame(groups[i])

In [9]:
R1_work=groups['R1_Work']

In [10]:
type(groups['R1_Work'])

pandas.core.frame.DataFrame

In [11]:
R1_work=R1_work.reset_index(drop=True)
R1_work

,Datetime,Sensor,State,Activity,Note
0,2009-02-02 14:29:03,M32,ON,R1_Work,begin
1,2009-02-02 17:37:22,M31,OFF,R1_Work,end
2,2009-02-02 19:00:18,M32,OFF,R1_Work,begin
3,2009-02-02 19:30:07,M31,OFF,R1_Work,end
4,2009-02-03 17:57:47,M30,ON,R1_Work,begin
...,...,...,...,...,...
113,2009-04-02 17:51:12,M32,ON,R1_Work,begin
114,2009-04-02 19:59:08,M31,OFF,R1_Work,end
115,2009-04-02 22:28:09,M31,OFF,R1_Work,end
116,2009-04-03 19:33:45,M32,OFF,R1_Work,begin


In [12]:
#Delete and change data.First remove the data describing the problem.
R1_work=R1_work.drop([114])
#R1_work=R1_work.loc[
R1_work.tail(10)

,Datetime,Sensor,State,Activity,Note
107,2009-03-30 18:33:20,M31,OFF,R1_Work,end
108,2009-03-31 08:59:38,M31,OFF,R1_Work,end
109,2009-03-31 21:04:08,M32,ON,R1_Work,begin
110,2009-03-31 22:41:02,M31,OFF,R1_Work,end
111,2009-04-02 08:01:06,M32,ON,R1_Work,begin
112,2009-04-02 08:01:57,M31,OFF,R1_Work,end
113,2009-04-02 17:51:12,M32,ON,R1_Work,begin
115,2009-04-02 22:28:09,M31,OFF,R1_Work,end
116,2009-04-03 19:33:45,M32,OFF,R1_Work,begin
117,2009-04-03 20:40:53,M32,OFF,R1_Work,end


In [13]:
#continue data cleaning
#R1_work.loc[1]=['2009-03-31  08:18:38', 'M31','ON','R1_Work','begin']
R1_work_add = pd.DataFrame({'Datetime':'2009-03-31  08:18:38', 'Sensor':'M31', 'State':'ON', 'Activity':'R1_Work', 'Note':'begin' },index=[0])
def insert(df, i, df_add):
    df1 = df.iloc[:i, :]
    df2 = df.iloc[i:, :]
    df_new = pd.concat([df1, df_add, df2], ignore_index=True)
    return df_new

R1_work = insert(R1_work, 108, R1_work_add)

In [14]:
R1_work.tail(15)

,Datetime,Sensor,State,Activity,Note
103,2009-03-28 01:34:04,M31,OFF,R1_Work,end
104,2009-03-30 16:08:19,M32,OFF,R1_Work,begin
105,2009-03-30 18:03:00,M31,OFF,R1_Work,end
106,2009-03-30 18:13:57,M31,OFF,R1_Work,begin
107,2009-03-30 18:33:20,M31,OFF,R1_Work,end
108,2009-03-31 08:18:38,M31,ON,R1_Work,begin
109,2009-03-31 08:59:38,M31,OFF,R1_Work,end
110,2009-03-31 21:04:08,M32,ON,R1_Work,begin
111,2009-03-31 22:41:02,M31,OFF,R1_Work,end
112,2009-04-02 08:01:06,M32,ON,R1_Work,begin


In [15]:
R1_work['Sensor'].value_counts()

M31    70
M32    38
M30     4
M33     2
M36     1
M37     1
M41     1
M45     1
Name: Sensor, dtype: int64

In [16]:
#R1_work['Sensor']['M45']
R1_work[R1_work['Sensor'].isin(['M45','M41'])]

,Datetime,Sensor,State,Activity,Note
12,2009-02-06 17:15:22,M41,ON,R1_Work,begin
13,2009-02-06 17:47:16,M45,OFF,R1_Work,end


In [17]:
#R1_work=R1_work.drop([12])
R1_work=R1_work.drop([13])
R1_work.head(15)

,Datetime,Sensor,State,Activity,Note
0,2009-02-02 14:29:03,M32,ON,R1_Work,begin
1,2009-02-02 17:37:22,M31,OFF,R1_Work,end
2,2009-02-02 19:00:18,M32,OFF,R1_Work,begin
3,2009-02-02 19:30:07,M31,OFF,R1_Work,end
4,2009-02-03 17:57:47,M30,ON,R1_Work,begin
5,2009-02-03 19:10:13,M31,ON,R1_Work,end
6,2009-02-04 21:05:38,M30,ON,R1_Work,begin
7,2009-02-04 22:48:19,M31,OFF,R1_Work,end
8,2009-02-05 18:38:09,M30,ON,R1_Work,begin
9,2009-02-05 21:50:56,M31,OFF,R1_Work,end


In [18]:
#reset index
R1_work=R1_work.reset_index(drop=True)

In [19]:
R1_work.head(15)

,Datetime,Sensor,State,Activity,Note
0,2009-02-02 14:29:03,M32,ON,R1_Work,begin
1,2009-02-02 17:37:22,M31,OFF,R1_Work,end
2,2009-02-02 19:00:18,M32,OFF,R1_Work,begin
3,2009-02-02 19:30:07,M31,OFF,R1_Work,end
4,2009-02-03 17:57:47,M30,ON,R1_Work,begin
5,2009-02-03 19:10:13,M31,ON,R1_Work,end
6,2009-02-04 21:05:38,M30,ON,R1_Work,begin
7,2009-02-04 22:48:19,M31,OFF,R1_Work,end
8,2009-02-05 18:38:09,M30,ON,R1_Work,begin
9,2009-02-05 21:50:56,M31,OFF,R1_Work,end


In [20]:
R1_work[R1_work['Sensor'].isin(['M30','M33','M36','M37'])]

,Datetime,Sensor,State,Activity,Note
4,2009-02-03 17:57:47,M30,ON,R1_Work,begin
6,2009-02-04 21:05:38,M30,ON,R1_Work,begin
8,2009-02-05 18:38:09,M30,ON,R1_Work,begin
10,2009-02-06 14:28:42,M36,ON,R1_Work,begin
11,2009-02-06 14:56:21,M37,ON,R1_Work,end
17,2009-02-09 18:46:07,M33,ON,R1_Work,begin
18,2009-02-09 19:31:02,M33,OFF,R1_Work,end
20,2009-02-10 11:09:44,M30,ON,R1_Work,end


In [21]:
#Overall modification of problem data
#2009/2/3  17:57:52 M31 ON  2009/2/4  21:05:59 M31 ON
#2009/2/5  18:38:52 M31 ON
#2009/2/6  14:29:30 M31 ON  2009/2/6  14:56:15 M31 ON
#2009/2/9  18:53:58 M31 ON  2009/2/9  19:30:53 M31 OFF
#2009/2/10  11:08:15 M31 OFF
R1_work.loc[4]=['2009-02-03 17:57:52', 'M31','ON','R1_Work','begin']
R1_work.loc[6]=['2009-02-04 21:05:59', 'M31','ON','R1_Work','begin']
R1_work.loc[8]=['2009-02-05 18:38:52', 'M31','ON','R1_Work','begin']
R1_work.loc[10]=['2009-02-06 14:29:30', 'M31','ON','R1_Work','begin']
R1_work.loc[11]=['2009-02-06 14:56:15', 'M31','ON','R1_Work','end']
R1_work.loc[16]=['2009-02-09 18:53:58', 'M31','ON','R1_Work','begin']
R1_work.loc[17]=['2009-02-09 19:30:53', 'M31','OFF','R1_Work','end']
R1_work.loc[17]=['2009-02-10 11:08:15', 'M31','OFF','R1_Work','end']
R1_work.head(11)

,Datetime,Sensor,State,Activity,Note
0,2009-02-02 14:29:03,M32,ON,R1_Work,begin
1,2009-02-02 17:37:22,M31,OFF,R1_Work,end
2,2009-02-02 19:00:18,M32,OFF,R1_Work,begin
3,2009-02-02 19:30:07,M31,OFF,R1_Work,end
4,2009-02-03 17:57:52,M31,ON,R1_Work,begin
5,2009-02-03 19:10:13,M31,ON,R1_Work,end
6,2009-02-04 21:05:59,M31,ON,R1_Work,begin
7,2009-02-04 22:48:19,M31,OFF,R1_Work,end
8,2009-02-05 18:38:52,M31,ON,R1_Work,begin
9,2009-02-05 21:50:56,M31,OFF,R1_Work,end


In [22]:
R1_work['Sensor'].value_counts()

M31    76
M32    38
M41     1
M33     1
M30     1
Name: Sensor, dtype: int64

In [23]:
#Identify activities associated with non-core sensors and correct them.
R1_work[R1_work['Sensor'].isin(['M30','M33','M36','M37'])]

,Datetime,Sensor,State,Activity,Note
18,2009-02-09 19:31:02,M33,OFF,R1_Work,end
20,2009-02-10 11:09:44,M30,ON,R1_Work,end


In [24]:
R1_work.loc[17]=['2009-02-09 19:30:53', 'M31','OFF','R1_Work','end']
R1_work.loc[19]=['2009-02-10 11:08:15', 'M31','OFF','R1_Work','end']
R1_work['Sensor'].value_counts()

M31    76
M32    38
M41     1
M33     1
M30     1
Name: Sensor, dtype: int64

In [25]:
#Output the data set after data cleaning.
R1_work.to_csv('data/R1work_data.csv')